In [5]:
import diffusers
import torch
import sys
sys.path.append("../")
from DDPMPipelineDropout import DDPMPipeline

import utils

In [6]:
model_path = "ddpm-ffhq-128/DDPM-FFHQ-TEST_v1_nodp.pth"
experiment_name = "10D-10S-ne"
model, config = utils.load_model(model_path)
scheduler = diffusers.DDPMScheduler(num_train_timesteps=config.num_train_timestamps)
pipeline = DDPMPipeline(unet=model, scheduler=scheduler).to("cuda")

Loading failed, Trying to Load data parallel mode


In [ ]:
#Set Eval params here
config.bayesian_avg_range = (0, 1000)
config.bayesian_avg_samples = 1
config.__dict__

In [ ]:
fid_score, inception_score = utils.calculate_metrics(config, pipeline, batch_size=1, num_images=2)

In [ ]:
try :
    results = torch.load("../results.dict")
except:
    results = {}

if config.run_name in results:
    results[config.run_name]["fid_score"] = fid_score.item()
    results[config.run_name]["path"] = model_path
    results[config.run_name]["inception_score_mean"] = inception_score[0].item()
    results[config.run_name]["inception_score_std"] = inception_score[1].item()
    results[config.run_name]["config"] = config.__dict__
else:
    results[config.run_name] = {}
    results[config.run_name]["fid_score"] = fid_score.item()
    results[config.run_name]["path"] = model_path
    results[config.run_name]["inception_score_mean"] = inception_score[0].item()
    results[config.run_name]["inception_score_std"] = inception_score[1].item()
    results[config.run_name]["config"] = config.__dict__

torch.save(results, "../results.dict")

In [ ]:
config.eval_batch_size = 4
utils.ddpm_evaluate(config, 0, pipeline)

In [7]:
config.run_name

'DDPM-FFHQ-TEST_v1_nodp'